In [20]:
%load_ext autoreload
%autoreload 2

%reload_ext autoreload


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# TO DO
## Structure
- [x] move classes out of main

## Logging
- [ ] change destination for log files - /var/log? - this may be handled by moving to a daemon model and allowing the system to manage logging
- [ ] filtering of log files filter based on source library?

## Configuration
- [ ] method for installing user config?
- [ ] script for installing as daemon

## Testing
- [ ] command line switches,

## Daemon
- [ ] implement daemon start/stop/restart features
- [ ] https://www.python.org/dev/peps/pep-3143 - instructions: https://dpbl.wordpress.com/2017/02/12/a-tutorial-on-python-daemon/
- [ ] restart on crash
- [ ] **This** is likely the best way forward: https://stackoverflow.com/questions/13069634/python-daemon-and-systemd-service

## Documentation
- [ ] README.md in epdlib module
- [ ] comsistently document attributes, methods 

## Feature Creep
- [ ] Clock that tells time as 'Quarter to Eight' or 'Half past Nine' or 'Ten after Seven'

## Notes
* Ubuntu Regular Font appears to truncate the last few characters with long strings in some situations; this does not appear to occur with other fonts. 



In [21]:
import logging
import logging.config

# parse arguments
import sys

# handle importing libraries based on config file
import importlib

# loop delay - sleep
from time import sleep

# clock
from datetime import datetime

# handle kill/interrupt singals
# import signal

# shell utilities for manpiulating files
# from shutil import copyfile, copyfileobj

##### PyPi Modules #####
# handle http requests
import requests

# rate limit the queries on the LMS server
from ratelimiter import RateLimiter

# create cache Path objects for managing temporary paths
# from cachepath import CachePath, TempPath, Path

# lmsquery-fork for managing comms with lms server
import lmsquery

In [22]:
import epdlib
from library import configuration
from library import signalhandler
from library import cacheart

In [23]:
def do_exit(status=0):
    if 'TESTING' in globals():
        if TESTING:
            logging.fatal(f'Exit called, but ignored due to global var `TESTING` = {TESTING}')
    else:
        sys.exit(status)

In [24]:
def time_now():
    '''return current time in HH:MM format
        e.g. 04:44, 23:16
    Returns:
        `str`'''
    return datetime.now().strftime("%H:%M")

In [25]:
# class SignalHandler(object):
#     '''handle specific signals and allow graceful exiting while loop
#         https://stackoverflow.com/a/31464349/5530152
    
#     Signals Handled Gracefully:
#         SIGINT
#         SIGTERM
#     Atributes:
#         kill_now (bool) default: False
#     '''
#     kill_now = False
#     def __init__(self):
#         signal.signal(signal.SIGINT, self.exit_gracefully)
#         signal.signal(signal.SIGTERM, self.exit_gracefully)
    
#     def exit_gracefully(self, signum, frame):
#         self.kill_now = True

In [26]:
# class cache_art():
#     '''download and cache album artwork from lms server
    
#     Args:
#         app_name(`str`): name of application - used to create /tmp/`app_name` directory
        
#     Properties:
#         app_name(`str`): name of application
#         cache_path(:obj:`CachePath`): path to store cached artwork
#         '''
#     def __init__(self, app_name):
#         self.app_name = app_name
        
#     @property
#     def app_name(self):
#         '''name of application
#             this is used to set the path in /tmp/`app_name`
            
#         Sets:
#             app_name(`str`): name of application
#             cache_path(obj:`CachePath`): path to store cached artwork'''
#         return self._app_name
    
#     @app_name.setter
#     def app_name(self, app_name):
#         self._app_name = app_name
#         self.cache_path = CachePath(app_name, dir=True)
    
#     def cache_artwork(self, artwork_url=None, album_id=None):
#         '''download and cache artwork as needed from lms server
        
#         Args:
#             artwork_url(`str`): URL of artwork on LMS server
#             album_id(`str`): unique string that identifies artwork on LMS server
            
#         Returns:
#             (obj:`pathlib.Path`): path to album artwork'''
#         if not artwork_url or not album_id:
#             raise TypeError(f'missing required value: artwork_url: {artwork_url}, album_id: {album_id}')
                
#         album_id = str(album_id)
        
#         artwork_path = self.cache_path/(album_id+'.jpg')
        
#         if artwork_path.exists():
#             logging.debug(f'artwork previously cached')
#             return artwork_path
        
#         r = False
#         try:
#             r = requests.get(artwork_url, stream=True)
#         except requests.exceptions.RequestException as e:
#             logging.error(f'failed to fetch artwork at: {artwork_url}: {e}')
        
#         if r:
#             try:
#                 with open(artwork_path, 'wb') as outFile:
#                     copyfileobj(r.raw, outFile)
#                     logging.debug(f'wrote ablum artwork to: {artwork_path}')
#             except (OSError, FileExistsError, ValueError) as e:
#                 logging.error(f'failed to write {artwork_path}')
#         else:
#             logging.error('failed to download album art due to previous errors')
#             return None
        
#         return artwork_path

#     def clear_cache(self, force=False):
#         '''clear the contents of the cache folder or wipe entirely
        
#         Args:
#             force(`bool`): False (default) - delete the files, True - delete entire directory and contents
#             '''
#         logging.debug(f'clearing previously downloaded files in {self.cache_path}')
#         if force:
#             logging.info(f'removing cache directory: {self.cache_path}')
#             try:
#                 self.cache_path.rm()
#             except Exception as e:
#                 logging.error(f'Error removing cach directory: {e}')
#                 return False
#         else:
#             try:
#                 self.cache_path.clear()
#             except Exception as e:
#                 logging.error(f'Error removing files in cache d directory: {e}')
#                 return False
            
#         return True

In [41]:
def main():
    import constants
    
    ########## CONSTANTS #########
    version = constants.version
    app_name = constants.app_name
    app_long_name = constants.app_long_name
    url = constants.url
    logging_cfg = configuration.fullPath(constants.logging_cfg)
    default_cfg = constants.default_cfg
    system_cfg = constants.system_cfg
    user_cfg = configuration.fullPath(constants.user_cfg)
    noartwork = configuration.fullPath(constants.noartwork)
    
    waveshare = constants.waveshare
    layouts_file = constants.layouts
    default_clock = constants.clock
    
    keyError_fmt = 'KeyError: section [{}] is missing value: {}'
    configError_fmt = 'see section [{}] in config file {}'
    valError_fmt = ''
    
    ######### CONFIGURATION #########
    
    ##### SETUP LOGGING #####
    logging.config.fileConfig(logging_cfg)
#     logger = logging.getLogger(__name__)
    
    ##### PARSE COMMAND LINE ARGUMENTS #####
    options = configuration.Options(sys.argv)
    # add options to the configuration object
    # for options that override the configuration file options, add in the format: 
    # dest=[[ConfigFileSectionName]]__[[Option_Name]]
    # options that are not in the configuraton file can be added using the format:
    # dest=commandline_option_name
    
    # set logging level
    options.add_argument('-l', '--log-level', ignore_none=True, 
                         type=str, choices=['DEBUG', 'INFO', 'WARNING', 'ERROR'], 
                         dest='main__log_level', 
                         help='set logging level: DEBUG, INFO, WARNING, ERROR')

    # alternative user_cfg file
    options.add_argument('-c', '--config', type=str, required=False, 
                         dest='user_cfg', ignore_none=True, default=user_cfg,
                         help=f'use the specified configuration file; default user config: {user_cfg}')
    
    # list servers - 
    options.add_argument('-s', '--list-servers', action='store_true', 
                         dest='list_servers',
                         default=False,
                         help='list servers and players found on local network and exit')
    
    # parse the command line options
    options.parse_args()

    ##### ACTION COMMANDLINE ARGS #####
    if options.options.list_servers:
        logging.error('FIXME - need to addd routine to list server info')
        do_exit(0)
        pass    
    
    # use an alternative user configuration file
    if 'user_cfg' in options.opts_dict:
        user_cfg = options.opts_dict['user_cfg']
    
    # read the configuration right most values overwrite left values
    # system overwrites default; user overwrites system
    config_file = configuration.ConfigFile(config_files=[default_cfg, system_cfg, user_cfg])    
    
    # merge the configuration files and the command line options
    config = configuration.merge_dict(config_file.config_dict, options.nested_opts_dict)
    
    # set root log level now
    ll = config['main']['log_level']
    logging.warning(f'log level is {ll}')
    logging.root.setLevel(ll)

    
    ##### OBJECTS #####
    # Signal handler for gracefully handling HUP/KILL signals
    sigHandler = signalhandler.SignalHandler()
    
    # LMS Query rate limiter wrapper - allow max of `max_calls` per `period` (seconds)
    lmsQuery_ratelimit = RateLimiter(max_calls=1, period=3)
    # create lms query object
    try:
        lms = lmsquery.LMSQuery(**config['lms_server'])
        if not lms.player_id:
            raise ValueError(keyError_fmt.format('lms_server', 'player_id'))
    except TypeError as e:
        logging.fatal(configError_fmt.format('lms_server', user_cfg))
        logging.fatal(f'Error: {e}')
        do_exit(0)
        
    except ValueError as e:
        logging.fatal(e)
        logging.fatal(f'locate server and player information with:\n$ {app_name} --list-servers')
        do_exit(0)
    
    # setup EPD Display
    try:
        epd = importlib.import_module('.'.join([waveshare, config['layouts']['display']]))
    except (KeyError) as e:
        logging.fatal(keyError_fmt.format('layouts', 'display'))
        logging.fatal(configError_fmt.format('layouts', user_cfg))
        logging.fatal(e)
        do_exit(0)
    except (ModuleNotFoundError) as e:
        logging.fatal(keyError_fmt.format('layouts', 'display'))
        logging.fatal(configError_fmt.format('layouts', user_cfg))
        logging.fatal(e)
        do_exit(0)
    
    # import additonal modules
    try:
        clock = importlib.import_module(config['modules']['clock'])
    except KeyError as e:
            logging.error(keyError_fmt.format('modules', 'clock'))
    except (ModuleNotFoundError) as e:
        mod = config['modules']['clock']
        logging.error(keyError_fmt.format('modules', 'clock'))
        logging.error(f'could not load module: {mod} due to error: {e}')
        logging.error('falling back to default')
        try:
            clock = importlib.import_module(default_clock)
        except ModuleNotFoundError as e:
            logging.fatal(f'failed to load module with error: {e}')
            
    try:
        clock_update = int(config['modules']['clock_update'])
    except KeyError as e:
        logging.error(keyError_fmt.format('modules', 'clock_update'))
        logging.error(f'setting clock update to 60 seconds')
        clock_update = 60
    
    # setup layouts for displaying content
    try:
        layouts = importlib.import_module(layouts_file)
        playing_layout_format = getattr(layouts, config['layouts']['now_playing'])
        stopped_layout_format = getattr(layouts, config['layouts']['stopped'])
        splash_layout_format = getattr(layouts, config['layouts']['splash'])
    except (ModuleNotFoundError) as e:
        logging.fatal(f'could not import layouts file: {layouts_file}')
        logging.fatal(e)
        do_exit(0)
    except (KeyError, AttributeError) as e:
        logging.fatal(keyError_fmt.format('layouts', e.args[0]))
        logging.fatal(configError_fmt.format('layouts', user_cfg))
        logging.fatal(e)
        do_exit(0)
    
    # set resolution for screen
    resolution = [epd.EPD_HEIGHT, epd.EPD_WIDTH]
    # sort to put longest dimension first for landscape layout
    resolution.sort(reverse=True)
    
    playing_layout = epdlib.Layout(layout=playing_layout_format, resolution=resolution)
    
    
    splash_layout = epdlib.Layout(layout=splash_layout_format, resolution=resolution)
    splash_layout.update_contents({'app_name': app_name,
                                   'version': version,
                                   'url': url})
    
    playing_layout = epdlib.Layout(layout=playing_layout_format, resolution=resolution)
    stopped_layout = epdlib.Layout(layout=stopped_layout_format, resolution=resolution)
    
    # scren objects for managing writing to screen
    screen = epdlib.Screen()
    screen.epd = epd.EPD()
    screen.initEPD()
    
    
    ########## EXECUTION ##########
    # Show splash screen
    logging.debug(f'starting up with this configuration: {config}')
    if config['main']['splash_screen']:
        # push the images in the layout to the screen
        screen.elements = splash_layout.blocks.values()
        # concat all the individual images
        screen.concat()
        # write the image
        screen.writeEPD()
    
    # refresh the screen when true
    refresh = False
    # maximum amount of time to wait before refreshing display
    refresh_delay = 60
    
    # vars for managing track ID, mode, album art
    nowplaying_id = None
    nowplaying_mode = None
    artwork_cache = cacheart.CacheArt(app_long_name)
    
    # loop forever waiting for a kill/interrupt signal
    try:
        while not sigHandler.kill_now:
            # clear the lms server response 
            response = None
        
            # use the ratelimiter to throttle requests
            with lmsQuery_ratelimit:
                try:
                    logging.debug(f'querying lms server for status of player: {lms.player_id}')
                    response = lms.now_playing()
                except requests.exceptions.ConnectionError as e:
                    logging.warning(f'Server could not find active player_id: {lms.player_id}')
                    logging.warning('is the specified player active?')
                    response = {'title': f'Could not connect player: {lms.player_id}',
                                'album': 'is player_id valid?',
                                'artist': 'see logs for more info',
                                'id': 'NONE', 'mode': 'ERROR - SEE LOGS'}
                    nowplaying_mode = response['mode']

                except KeyError as e:
                    logging.info(f'No playlist is active on player_id: {lms.player_id}')
                    response = {'title': 'No music is queued', 'id': 'NONE', 'mode': 'No Playlist'}
                    nowplaying_mode = response['mode']
                    
            if response:
                resp_id = response['id']
                resp_mode = response['mode']
                if resp_id != nowplaying_id or resp_mode != nowplaying_mode:
                    logging.info(f'track/mode change {resp_mode}')
                    nowplaying_id = resp_id
                    nowplaying_mode = resp_mode
                    
                    # add the path to the album art into the response
                    try:
                        artwork = artwork_cache.cache_artwork(response['artwork_url'], response['album_id'])
                    except KeyError as e:
                        logging.warning('no artwork available')
                        artwork = None
                    if not artwork:
                        artwork = noartwork
                    response['coverart'] = str(artwork)
                             
                    playing_layout.update_contents(response)
                    refresh = playing_layout
                    #set delay to 60 seconds
                    refresh_delay = 60
                else:
                    refresh = False
                
                

#             if nowplaying_mode != 'play' and screen.update.last_updated > 60:
#                 logging.debug('music appears to be paused')
#                 stopped_layout.update_contents({'time': time_now(), 'mode': nowplaying_mode})
#                 refresh = stopped_layout
            if nowplaying_mode != "play" and screen.update.last_updated > refresh_delay:
                logging.debug(f'next update will be in {refresh_delay} seconds')
            
                logging.debug('music appears to be paused')
                stopped_layout.update_contents(clock.get_time())
#                 stopped_layout.update_contents({'time': time_now(), 'mode': nowplaying_mode})
                refresh = stopped_layout
                refresh_delay = clock_update


        
            # only refresh if needed
            if refresh:
                logging.debug('refreshing display')
                screen.initEPD()
                screen.elements=refresh.blocks.values()
                screen.concat()
                screen.writeEPD()   
                refresh = False                            
                    
    
            sleep(0.5)
    finally:
        logging.info('cleaning up and wiping screen')
        artwork_cache.clear_cache()
        
#         screen.initEPD()
#         screen.clearEPD()
        
    return config

In [ ]:
TESTING = True
c = main()

configuration:parse_args:154:WARNING - ignoring unknown options: ['-f', '/home/pi/.local/share/jupyter/runtime/kernel-bcd34a93-049b-4b54-82fc-01dad72fccde.json']
configuration:parse_config:389:WARNING - /etc/slimpi.cfg does not exist
<ipython-input-41-2f0ceabf752b>:main:76:WARNING - log level is DEBUG
Layout:__init__:93:DEBUG - no font specified yet
Layout:layout:200:DEBUG - calculating values from layout for resolution [600, 448]
Layout:_calculate_layout:233:DEBUG - ***title***
Layout:_check_keys:109:DEBUG - checking key/values
Layout:_check_keys:114:DEBUG - missing key: rand; adding and setting to False
Layout:_check_keys:114:DEBUG - missing key: inverse; adding and setting to False
Layout:_check_keys:114:DEBUG - missing key: maxchar; adding and setting to None
Layout:_check_keys:114:DEBUG - missing key: dimensions; adding and setting to None
Layout:_calculate_layout:240:DEBUG - dimensions: (600, 256)
Layout:_calculate_layout:269:DEBUG - has explict position
Layout:_calculate_layout:

Block:_text2image:389:DEBUG - max x dim so far: 4
Block:_text2image:393:DEBUG - dimensions of text portion of image: (4, 22)
Block:_text2image:403:DEBUG - drawing text at 0, 0
Block:_text2image:404:DEBUG - with dimensions: 4, 22
Block:_text2image:430:DEBUG - pasting text portion at coordinates: 0, 21
Layout:_set_images:299:DEBUG - ***mode***)
Layout:_set_images:303:DEBUG - set text block
Block:area:97:DEBUG - maximum area: (360, 32)
Block:inverse:76:DEBUG - set inverse: False
Block:abs_coordinates:115:DEBUG - absolute coordinates: (240, 416)
Block:__init__:246:DEBUG - create TextBlock
Block:maxchar:323:DEBUG - maximum characters per line: 42
Block:text_formatter:366:DEBUG - formatted list:
 ['.']
Block:_text2image:370:DEBUG - creating blank image area: (360, 32) with inverse: False
Block:_text2image:385:DEBUG - line size: 4, 22
Block:_text2image:389:DEBUG - max x dim so far: 4
Block:_text2image:393:DEBUG - dimensions of text portion of image: (4, 22)
Block:_text2image:403:DEBUG - drawi

Block:_text2image:385:DEBUG - line size: 405, 34
Block:_text2image:389:DEBUG - max x dim so far: 405
Block:_text2image:393:DEBUG - dimensions of text portion of image: (405, 34)
Block:_text2image:401:DEBUG - hcenter line: https://github.com/txoof/slimpi_epd
Block:_text2image:403:DEBUG - drawing text at 0, 0
Block:_text2image:404:DEBUG - with dimensions: 405, 34
Block:_text2image:430:DEBUG - pasting text portion at coordinates: 98, 50
Layout:__init__:93:DEBUG - no font specified yet
Layout:layout:200:DEBUG - calculating values from layout for resolution [600, 448]
Layout:_calculate_layout:233:DEBUG - ***title***
Layout:_check_keys:109:DEBUG - checking key/values
Layout:_check_keys:114:DEBUG - missing key: rand; adding and setting to False
Layout:_check_keys:114:DEBUG - missing key: inverse; adding and setting to False
Layout:_check_keys:114:DEBUG - missing key: maxchar; adding and setting to None
Layout:_check_keys:114:DEBUG - missing key: dimensions; adding and setting to None
Layout:_

Block:_text2image:370:DEBUG - creating blank image area: (360, 64) with inverse: False
Block:_text2image:385:DEBUG - line size: 4, 22
Block:_text2image:389:DEBUG - max x dim so far: 4
Block:_text2image:393:DEBUG - dimensions of text portion of image: (4, 22)
Block:_text2image:403:DEBUG - drawing text at 0, 0
Block:_text2image:404:DEBUG - with dimensions: 4, 22
Block:_text2image:430:DEBUG - pasting text portion at coordinates: 0, 21
Layout:_set_images:299:DEBUG - ***mode***)
Layout:_set_images:303:DEBUG - set text block
Block:area:97:DEBUG - maximum area: (360, 32)
Block:inverse:76:DEBUG - set inverse: False
Block:abs_coordinates:115:DEBUG - absolute coordinates: (240, 416)
Block:__init__:246:DEBUG - create TextBlock
Block:maxchar:323:DEBUG - maximum characters per line: 42
Block:text_formatter:366:DEBUG - formatted list:
 ['.']
Block:_text2image:370:DEBUG - creating blank image area: (360, 32) with inverse: False
Block:_text2image:385:DEBUG - line size: 4, 22
Block:_text2image:389:DEBU

epdconfig:module_exit:80:DEBUG - spi end
epdconfig:module_exit:83:DEBUG - close 5V, Module enters 0 power consumption ...
<ipython-input-41-2f0ceabf752b>:main:205:DEBUG - querying lms server for status of player: dc:a6:32:29:99:ef
<ipython-input-41-2f0ceabf752b>:main:225:INFO - track/mode change pause
cacheart:cache_artwork:64:DEBUG - wrote ablum artwork to: /tmp/com.txoof.slimpi/5573.jpg
Layout:update_contents:332:DEBUG - updating blocks
Layout:update_contents:341:DEBUG - ignoring block id
Layout:update_contents:338:DEBUG - updating block: title
Block:text_formatter:366:DEBUG - formatted list:
 ["It's No Use"]
Block:_text2image:370:DEBUG - creating blank image area: (600, 256) with inverse: False
Block:_text2image:385:DEBUG - line size: 322, 90
Block:_text2image:389:DEBUG - max x dim so far: 322
Block:_text2image:393:DEBUG - dimensions of text portion of image: (322, 90)
Block:_text2image:401:DEBUG - hcenter line: It's No Use
Block:_text2image:403:DEBUG - drawing text at 0, 0
Block:_t

epd5in83:ReadBusy:69:DEBUG - e-Paper busy
epd5in83:ReadBusy:72:DEBUG - e-Paper busy release
epd5in83:ReadBusy:69:DEBUG - e-Paper busy
epd5in83:ReadBusy:72:DEBUG - e-Paper busy release
epdconfig:module_exit:80:DEBUG - spi end
epdconfig:module_exit:83:DEBUG - close 5V, Module enters 0 power consumption ...
<ipython-input-41-2f0ceabf752b>:main:205:DEBUG - querying lms server for status of player: dc:a6:32:29:99:ef
<ipython-input-41-2f0ceabf752b>:main:205:DEBUG - querying lms server for status of player: dc:a6:32:29:99:ef
<ipython-input-41-2f0ceabf752b>:main:205:DEBUG - querying lms server for status of player: dc:a6:32:29:99:ef
<ipython-input-41-2f0ceabf752b>:main:205:DEBUG - querying lms server for status of player: dc:a6:32:29:99:ef
<ipython-input-41-2f0ceabf752b>:main:205:DEBUG - querying lms server for status of player: dc:a6:32:29:99:ef
<ipython-input-41-2f0ceabf752b>:main:205:DEBUG - querying lms server for status of player: dc:a6:32:29:99:ef
<ipython-input-41-2f0ceabf752b>:main:205

In [ ]:
c